<a href="https://colab.research.google.com/github/marcelortizv/JELcodes_prediction/blob/main/03_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 1.3 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180943 sha256=ee3acfa0f57f9608b5914ad505b028032357ed262136fb3a90178f71c17258f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-u50ibprx/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [4]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    2.2.4                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.11                        
Models           en                            



In [5]:
import spacy 
import en_core_web_lg
nlp = en_core_web_lg.load()

In [8]:
x = 'king man woman'
doc = nlp(x)

In [9]:
for token1 in doc:
  for token2 in doc:
    print(token1.text, token2.text, token1.similarity(token2))

king king 1.0
king man 0.4088461
king woman 0.26556593
man king 0.4088461
man man 1.0
man woman 0.7401745
woman king 0.26556593
woman man 0.7401745
woman woman 1.0


### Model Building

In [13]:
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.multiclass import OneVsRestClassifier

import ast

In [43]:
def Accuracy(y_true, y_pred):
    """
    Accuracy based on Jaccard Similarity Score
    :param y_true: ground truth
    :param y_pred: prediction
    :return: Jaccard Similarity Score
    """
    jaccard = np.minimum(y_true, y_pred).sum(axis=1) / np.maximum(y_true, y_pred).sum(axis=1)
    return jaccard.mean()


def print_ml_score(y_test, y_pred, clf):
    print('Classifier: ', clf.__class__.__name__)
    print('Accuracy Score: {}'.format(Accuracy(y_test, y_pred)))
    print("-----------------------------------")


def train_model(classifier, feature_vector_train, label_train, feature_vector_test, label_test):
    # fit the training set on the classifier
    clf = OneVsRestClassifier(classifier)
    clf.fit(feature_vector_train, label_train)

    # predict the labels on test set
    predictions = clf.predict(feature_vector_test)

    return print_ml_score(label_test, predictions, classifier)

def get_vec(x):
    doc = nlp(x)
    vec = doc.vector
    return vec

In [14]:
df = pd.read_csv('/content/traning_data_cleaned_v02.csv')
df['tags'] = df['tags'].apply(lambda x: ast.literal_eval(x))
print(df.shape)
df.head()

(3126, 6)


,abstract,tags,title_x,all_text,cleaned_abstract,cleaned_all_text
0,When a production process requires two extreme...,"[O, D, G, E]",optimal adoption of complementary technologies,optimal adoption of complementary technologies...,production process require two extremely compl...,optimal adoption complementary technologies pr...
1,The Japanese banking crisis provides a natural...,"[G, E]",collateral damage: effects of the japanese ban...,collateral damage: effects of the japanese ban...,japanese bank crisis provide natural experimen...,collateral damage effect japanese bank crisis...
2,We consider a market with red and green worker...,[J],endogenous inequality in integrated labor mark...,endogenous inequality in integrated labor mark...,consider market red green workers label payof...,endogenous inequality integrate labor market t...
3,This paper presents a general-equilibrium mode...,"[J, R]","labor-market integration, investment in risky ...","labor-market integration, investment in risky ...",present general-equilibrium human capital inve...,labor-market integration investment risky hum...
4,This paper develops a theory of inequality and...,"[P, E, I, D]",unequal societies: income distribution and the...,unequal societies: income distribution and the...,develop theory inequality social contract aim ...,unequal societies income distribution social ...


### Word2vec

In [26]:
df['vec'] = df['cleaned_all_text'].apply(lambda x: get_vec(x))

In [29]:
# Reshaping
X = df['vec'].to_numpy()
X = X.reshape(-1,1)

X = np.concatenate(np.concatenate(X, axis = 0), axis = 0).reshape(-1, 300)
print(X.shape)

(3126, 300)


In [30]:

# Convert outcome variable to one-hot encoding type
multilabel = MultiLabelBinarizer()
y = multilabel.fit_transform(df['tags'])

In [31]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=123)

### Training Model

In [46]:
# set models
sgd = SGDClassifier()
lr = LogisticRegression(solver = 'lbfgs')
svc = LinearSVC()
rf = RandomForestClassifier()
#naive = MultinomialNB()
xgboost = XGBClassifier()

In [47]:
# training using word2vec
for classifier in [sgd, lr, svc, rf, xgboost]:
    train_model(classifier, xtrain, ytrain, xtest, ytest)

/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


Classifier:  SGDClassifier
Accuracy Score: 0.48912977331507684
-----------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


Classifier:  LogisticRegression
Accuracy Score: 0.48498402555910547
-----------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


Classifier:  LinearSVC
Accuracy Score: 0.5210748516659061
-----------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


Classifier:  RandomForestClassifier
Accuracy Score: 0.35921192758253456
-----------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


Classifier:  XGBClassifier
Accuracy Score: 0.47263045793397224
-----------------------------------


#### Tuning Linear SVC Model

In [59]:
from sklearn import svm
svc = svm.SVC()
param_grid = {"estimator__C": [0.1,1,2], "estimator__gamma": [1,0.1,0.01,0.001], "estimator__kernel": ['rbf', 'poly', 'sigmoid']}

In [60]:
clf = OneVsRestClassifier(svc)
model_tunning = GridSearchCV(clf, param_grid=param_grid, refit=True,verbose=2)
model_tunning.fit(xtrain, ytrain)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV] estimator__C=0.1, estimator__gamma=1, estimator__kernel=rbf .....


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


[CV]  estimator__C=0.1, estimator__gamma=1, estimator__kernel=rbf, total=  16.6s
[CV] estimator__C=0.1, estimator__gamma=1, estimator__kernel=rbf .....


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.6s remaining:    0.0s
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


[CV]  estimator__C=0.1, estimator__gamma=1, estimator__kernel=rbf, total=  16.5s
[CV] estimator__C=0.1, estimator__gamma=1, estimator__kernel=rbf .....


...

[CV]  estimator__C=2, estimator__gamma=0.001, estimator__kernel=sigmoid, total=  12.2s
[CV] estimator__C=2, estimator__gamma=0.001, estimator__kernel=sigmoid 


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


[CV]  estimator__C=2, estimator__gamma=0.001, estimator__kernel=sigmoid, total=  12.1s


[Parallel(n_jobs=1)]: Done 180 out of 180 | elapsed: 37.6min finished
/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


GridSearchCV(cv=None, error_score=nan,
             estimator=OneVsRestClassifier(estimator=SVC(C=1.0,
                                                         break_ties=False,
                                                         cache_size=200,
                                                         class_weight=None,
                                                         coef0=0.0,
                                                         decision_function_shape='ovr',
                                                         degree=3,
                                                         gamma='scale',
                                                         kernel='rbf',
                                                         max_iter=-1,
                                                         probability=False,
                                                         random_state=None,
                                                         shrinking=True,
            

In [61]:
print(model_tunning.best_score_)
print(model_tunning.best_params_)

0.2584
{'estimator__C': 2, 'estimator__gamma': 1, 'estimator__kernel': 'rbf'}


In [62]:
predictions = model_tunning.predict(xtest)

print_ml_score(ytest, predictions, svc)

Classifier:  SVC
Accuracy Score: 0.5394188346265024
-----------------------------------


In [67]:
predictions

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [63]:
# saving model
import pickle
pickle.dump(model_tunning, open("svc_tuned_w2v.pkl", "wb"))

In [37]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [38]:
# set models 2
adaboost = AdaBoostClassifier()
bagging = BaggingClassifier()
extra = ExtraTreesClassifier()
tree = DecisionTreeClassifier()
mlp = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [39]:
# training using word2vec
for classifier in [adaboost, bagging, extra, tree, mlp]:
    train_model(classifier, xtrain, ytrain, xtest, ytest)

/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


Classifier:  AdaBoostClassifier
Accuracy Score: 0.44445839038490786
-----------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


Classifier:  BaggingClassifier
Accuracy Score: 0.35958466453674126
-----------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


Classifier:  ExtraTreesClassifier
Accuracy Score: 0.3498402555910543
-----------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))


Classifier:  DecisionTreeClassifier
Accuracy Score: 0.3105412292712612
-----------------------------------
Classifier:  MLPClassifier
Accuracy Score: 0.0
-----------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/multiclass.py:75: UserWarning: Label not 18 is present in all training examples.
  str(classes[c]))
